In [91]:
import os 
import sys


import FINE as fn
import FINE.IOManagement.xarrayIO as xrIO 
import pandas as pd

# %load_ext autoreload
# %autoreload 2

# How to save an energy system model instance and set it back up? 

**Xarray and NetCDF files to the rescue!** The data contained within an Energy System Model (ESM) instance is vast and complex. Saving it directly is not possible. It can, however, be saved as a NetCDF file which supports complex data structures. 

#### What exactly is NetCDF? 
NetCDF (Network Common Data Format) is a set of software libraries and machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. It is also a community standard for sharing scientific data. 

#### Python modules that support working with NetCDF files:
1. netcdf4-python: Official Python interface to netCDF files
2. PyNIO: To access different file formats such as netCDF, HDF, and GRIB
3. xarray: Based on NumPy and pandas

Note: xarray module is used here. 

For our use case, the following functionalities are provided: 
* Conversion of ESM instance to xarray dataset. Additionally, possible to save this dataset as NetCDF file in a desired folder, with a desired file name. 
* Conversion of xarray dataset/saved NetCDF file back to ESM instance.

#### Structure of the xarray dataset: 

<img src="xarray_fine.png" style="width: 1000px;"/>


## Conversion of ESM instance to xarray dataset and saving it as a NetCDF file

#### STEP 1. Set up your  ESM instance 

In [92]:
from getModel import getModel
esM = getModel()
esM_input = getModel()
esM.optimize()

Using license file C:\Users\p.dunkel\gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file C:\Users\P2C3B~1.DUN\AppData\Local\Temp\tmpi_f8l8t0.pyomo.lp
Reading time = 0.01 seconds
x103: 91 rows, 63 columns, 221 nonzeros
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter Threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Parameter logfile unchanged
   Value:   Default: 
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 91 rows, 63 columns and 221 nonzeros
Model fingerprint: 0xc4afb60b
Coefficient statistics:
  Matrix range     [3e-01, 2e+03]
  Objective range  [7e-03, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 41 rows and 29 columns
Presolve time: 0.00s
Presolved: 50 rows, 34 columns, 168 nonzeros

Iteration    Ob

#### STEP 2. Conversion to xarray datasets and saving as NetCDF file
You can convert the esM to xarray datasets with `esm_to_datasets` and access Input, Parameters or Result.


In [93]:
import warnings
warnings.filterwarnings('ignore')
esm_datasets = xrIO.esm_to_datasets(esM)
esm2 = xrIO.datasets_to_esm(esm_datasets)

In [94]:
esM.getOptimizationSummary('SourceSinkModel').index.get_level_values(0).unique()

Index(['Electricity market', 'sink_hydrogen'], dtype='object', name='Component')

In [95]:
esm2.componentModelingDict['SourceSinkModel'].getOptimalValues()

{'capacityVariablesOptimum': {'values': None,
  'timeDependent': False,
  'dimension': '1dim'},
 'isBuiltVariablesOptimum': {'values': None,
  'timeDependent': False,
  'dimension': '1dim'},
 'operationVariablesOptimum': {'values': None,
  'timeDependent': True,
  'dimension': '1dim'}}

In [96]:
xr_dss = esm_datasets
for model, comps  in xr_dss['Results'].items():
    # read optSummary
    optSummary = comps['optSummary']
    df = pd.DataFrame([])
    for item in optSummary:
        _df = optSummary[item].to_dataframe().T
        iterables = [[item,key,value] for key,value in optSummary[item].attrs.items()]
        _df.index = pd.MultiIndex.from_tuples(iterables)
        _df.index.names = ['Component','Property','Unit']
        df = df.append(_df)
    break
df
model

'SourceSinkModel'

In [97]:
comps = xr_dss['Results']['SourceSinkModel']

for component in comps['optValues']:
    aa = comps['optValues'][component]
    variable = list(aa.keys())
    #variable_valu
    #print(aa)    
#aa.to_dataframe()
aa['operationVariablesOptimum'].values

array([[13140000.],
       [13140000.],
       [13140000.],
       [13140000.]])

In [128]:
aa = comps['optValues']['Electricity market']
aa
component = 'Electricity market'
space = aa.coords
space.get('space').values
comps['optValues']

{'Electricity market': <xarray.Dataset>
 Dimensions:                    (space: 1, time: 4)
 Coordinates:
   * time                       (time) int64 0 1 2 3
   * space                      (space) <U21 'Electrolyzer_Location'
 Data variables:
     operationVariablesOptimum  (time, space) float64 1.877e+07 ... 1.877e+07,
 'sink_hydrogen': <xarray.Dataset>
 Dimensions:                    (space: 1, time: 4)
 Coordinates:
   * time                       (time) int64 0 1 2 3
   * space                      (space) <U16 'IndustryLocation'
 Data variables:
     operationVariablesOptimum  (time, space) float64 1.314e+07 ... 1.314e+07}

In [132]:
esM.componentModelingDict['SourceSinkModel'].getOptimalValues().keys()

dict_keys(['capacityVariablesOptimum', 'isBuiltVariablesOptimum', 'operationVariablesOptimum'])

In [109]:
getattr(esM.componentModelingDict['SourceSinkModel'],'operationVariablesOptimum','')
#setattr(esM.componentModelingDict['SourceSinkModel'],'operationVariablesOptimum','')

,,0,1,2,3
Electricity market,Electrolyzer_Location,1.877143e+07,3.754286e+07,0.0,1.877143e+07
sink_hydrogen,IndustryLocation,1.314000e+07,1.314000e+07,13140000.0,1.314000e+07


In [100]:
esM.componentModelingDict['SourceSinkModel'].getOptimalValues()['operationVariablesOptimum']['values']

,,0,1,2,3
Electricity market,Electrolyzer_Location,1.877143e+07,3.754286e+07,0.0,1.877143e+07
sink_hydrogen,IndustryLocation,1.314000e+07,1.314000e+07,13140000.0,1.314000e+07


In [101]:
pd.to_numeric(esM.getOptimizationSummary('SourceSinkModel',0).loc['Electricity market','capacity'].iloc[-1])

space
Electrolyzer_Location   NaN
IndustryLocation        NaN
Name: [kW$_{el}$], dtype: float64

In [102]:
esm_datasets['Results']['SourceSinkModel']['optSummary']['Electricity market']

<xarray.Dataset>
Dimensions:         (space: 2)
Coordinates:
  * space           (space) object 'Electrolyzer_Location' 'IndustryLocation'
Data variables:
    TAC             (space) float64 1.52e+06 0.0
    capacity        (space) float64 nan nan
    capexCap        (space) float64 nan nan
    capexIfBuilt    (space) float64 nan nan
    commodCosts     (space) float64 1.896e+06 nan
    commodRevenues  (space) float64 3.754e+05 nan
    invest          (space) float64 nan nan
    isBuilt         (space) float64 nan nan
    operation       (space) float64 7.509e+07 nan
    opexCap         (space) float64 nan nan
    opexIfBuilt     (space) float64 nan nan
    opexOp          (space) float64 0.0 nan
Attributes:
    TAC:             [1 Euro/a]
    capacity:        [kW$_{el}$]
    capexCap:        [1 Euro/a]
    capexIfBuilt:    [1 Euro/a]
    commodCosts:     [1 Euro/a]
    commodRevenues:  [1 Euro/a]
    invest:          [1 Euro]
    isBuilt:         [-]
    operation:       [kW$_{el}$*h]
    opexCap:         [1 Euro/a]
    opexIfBuilt:     [1 Euro/a]
    opexOp:          [1 Euro/a]

In [103]:
esM.componentModelingDict['SourceSinkModel'].optSummary.loc['Electricity market'].index.get_level_values(0)

Index(['TAC', 'capacity', 'capexCap', 'capexIfBuilt', 'commodCosts',
       'commodRevenues', 'invest', 'isBuilt', 'operation', 'operation',
       'opexCap', 'opexIfBuilt', 'opexOp'],
      dtype='object', name='Property')

In [104]:
unit = list(esM.componentModelingDict['SourceSinkModel'].optSummary.index.get_level_values(2).unique())
property = list(esM.componentModelingDict['SourceSinkModel'].optSummary.index.get_level_values(1).unique())
xrd = esm_datasets['Results']['SourceSinkModel']['optSummary']['Electricity market']

asdf = dict(zip(property,unit))
xrd.attrs = dict(zip(property,unit))
xrd.attrs['test'] = 0
property

['TAC',
 'capacity',
 'capexCap',
 'capexIfBuilt',
 'commodCosts',
 'commodRevenues',
 'invest',
 'isBuilt',
 'operation',
 'opexCap',
 'opexIfBuilt',
 'opexOp']

In [105]:
unit

['[1 Euro/a]',
 '[kW$_{el}$]',
 '[1 Euro]',
 '[-]',
 '[kW$_{el}$*h/a]',
 '[kW$_{el}$*h]',
 '[kW$_{H_{2},LHV}$]',
 '[kW$_{H_{2},LHV}$*h/a]',
 '[kW$_{H_{2},LHV}$*h]']

In [106]:
esM.componentModelingDict['SourceSinkModel'].optSummary#.to_xarray()

space                                                    Electrolyzer_Location  \
Component          Property       Unit                                           
Electricity market TAC            [1 Euro/a]                       1.52049e+06   
                   capacity       [kW$_{el}$]                              NaN   
                   capexCap       [1 Euro/a]                               NaN   
                   capexIfBuilt   [1 Euro/a]                               NaN   
                   commodCosts    [1 Euro/a]                       1.89591e+06   
                   commodRevenues [1 Euro/a]                            375429   
                   invest         [1 Euro]                                 NaN   
                   isBuilt        [-]                                      NaN   
                   operation      [kW$_{el}$*h/a]                  7.50857e+07   
                                  [kW$_{el}$*h]                    7.50857e+07   
                   opexCap        [1 Euro/a]                               NaN   
                   opexIfBuilt    [1 Euro/a]                               NaN   
                   opexOp         [1 Euro/a]                                 0   
sink_hydrogen      TAC            [1 Euro/a]                                 0   
                   capacity       [kW$_{H_{2},LHV}$]                       NaN   
                   capexCap       [1 Euro/a]                               NaN   
                   capexIfBuilt   [1 Euro/a]                               NaN   
                   commodCosts    [1 Euro/a]                               NaN   
                   commodRevenues [1 Euro/a]                               NaN   
                   invest         [1 Euro]                                 NaN   
                   isBuilt        [-]                                      NaN   
                   operation      [kW$_{H_{2},LHV}$*h/a]                   NaN   
                                  [kW$_{H_{2},LHV}$*h]                     NaN   
                   opexCap        [1 Euro/a]                               NaN   
                   opexIfBuilt    [1 Euro/a]                               NaN   
                   opexOp         [1 Euro/a]                               NaN   

space                                                    IndustryLocation  
Component          Property       Unit                                     
Electricity market TAC            [1 Euro/a]                            0  
                   capacity       [kW$_{el}$]                         NaN  
                   capexCap       [1 Euro/a]                          NaN  
                   capexIfBuilt   [1 Euro/a]                          NaN  
                   commodCosts    [1 Euro/a]                          NaN  
                   commodRevenues [1 Euro/a]                          NaN  
                   invest         [1 Euro]                            NaN  
                   isBuilt        [-]                                 NaN  
                   operation      [kW$_{el}$*h/a]                     NaN  
                                  [kW$_{el}$*h]                       NaN  
                   opexCap        [1 Euro/a]                          NaN  
                   opexIfBuilt    [1 Euro/a]                          NaN  
                   opexOp         [1 Euro/a]                          NaN  
sink_hydrogen      TAC            [1 Euro/a]                            0  
                   capacity       [kW$_{H_{2},LHV}$]                  NaN  
                   capexCap       [1 Euro/a]                          NaN  
                   capexIfBuilt   [1 Euro/a]                          NaN  
                   commodCosts    [1 Euro/a]                            0  
                   commodRevenues [1 Euro/a]                            0  
                   invest         [1 Euro]                            NaN  
                   isBuilt     

In [107]:
esM2 = xrIO.datasets_to_esm(esm_datasets)
for model in esM2.componentModelingDict.keys():
    print(getattr(esM2.componentModelingDict[model],'optSummary'))

ValueError: Length mismatch: Expected axis has 12 elements, new values have 10 elements

In [ ]:
esM.getOptimizationSummary('SourceSinkModel', outputLevel=0)

space                                                    Electrolyzer_Location  \
Component          Property       Unit                                           
Electricity market TAC            [1 Euro/a]                       1.52049e+06   
                   capacity       [kW$_{el}$]                              NaN   
                   capexCap       [1 Euro/a]                               NaN   
                   capexIfBuilt   [1 Euro/a]                               NaN   
                   commodCosts    [1 Euro/a]                       1.89591e+06   
                   commodRevenues [1 Euro/a]                            375429   
                   invest         [1 Euro]                                 NaN   
                   isBuilt        [-]                                      NaN   
                   operation      [kW$_{el}$*h/a]                  7.50857e+07   
                                  [kW$_{el}$*h]                    7.50857e+07   
                   opexCap        [1 Euro/a]                               NaN   
                   opexIfBuilt    [1 Euro/a]                               NaN   
                   opexOp         [1 Euro/a]                                 0   
sink_hydrogen      TAC            [1 Euro/a]                                 0   
                   capacity       [kW$_{H_{2},LHV}$]                       NaN   
                   capexCap       [1 Euro/a]                               NaN   
                   capexIfBuilt   [1 Euro/a]                               NaN   
                   commodCosts    [1 Euro/a]                               NaN   
                   commodRevenues [1 Euro/a]                               NaN   
                   invest         [1 Euro]                                 NaN   
                   isBuilt        [-]                                      NaN   
                   operation      [kW$_{H_{2},LHV}$*h/a]                   NaN   
                                  [kW$_{H_{2},LHV}$*h]                     NaN   
                   opexCap        [1 Euro/a]                               NaN   
                   opexIfBuilt    [1 Euro/a]                               NaN   
                   opexOp         [1 Euro/a]                               NaN   

space                                                    IndustryLocation  
Component          Property       Unit                                     
Electricity market TAC            [1 Euro/a]                            0  
                   capacity       [kW$_{el}$]                         NaN  
                   capexCap       [1 Euro/a]                          NaN  
                   capexIfBuilt   [1 Euro/a]                          NaN  
                   commodCosts    [1 Euro/a]                          NaN  
                   commodRevenues [1 Euro/a]                          NaN  
                   invest         [1 Euro]                            NaN  
                   isBuilt        [-]                                 NaN  
                   operation      [kW$_{el}$*h/a]                     NaN  
                                  [kW$_{el}$*h]                       NaN  
                   opexCap        [1 Euro/a]                          NaN  
                   opexIfBuilt    [1 Euro/a]                          NaN  
                   opexOp         [1 Euro/a]                          NaN  
sink_hydrogen      TAC            [1 Euro/a]                            0  
                   capacity       [kW$_{H_{2},LHV}$]                  NaN  
                   capexCap       [1 Euro/a]                          NaN  
                   capexIfBuilt   [1 Euro/a]                          NaN  
                   commodCosts    [1 Euro/a]                            0  
                   commodRevenues [1 Euro/a]                            0  
                   invest         [1 Euro]                            NaN  
                   isBuilt     

In [ ]:
esM.componentModelingDict['SourceSinkModel'].getOptimalValues()

{'capacityVariablesOptimum': {'values': None,
  'timeDependent': False,
  'dimension': '1dim'},
 'isBuiltVariablesOptimum': {'values': None,
  'timeDependent': False,
  'dimension': '1dim'},
 'operationVariablesOptimum': {'values':                                                      0             1  \
  Electricity market Electrolyzer_Location  1.877143e+07  3.754286e+07   
  sink_hydrogen      IndustryLocation       1.314000e+07  1.314000e+07   
  
                                                     2             3  
  Electricity market Electrolyzer_Location         0.0  1.877143e+07  
  sink_hydrogen      IndustryLocation       13140000.0  1.314000e+07  ,
  'timeDependent': True,
  'dimension': '1dim'}}

In [ ]:
esM.componentModelingDict['SourceSinkModel'].getOptimalValues()['operationVariablesOptimum']['values']

,,0,1,2,3
Electricity market,Electrolyzer_Location,1.877143e+07,3.754286e+07,0.0,1.877143e+07
sink_hydrogen,IndustryLocation,1.314000e+07,1.314000e+07,13140000.0,1.314000e+07


In [ ]:
setattr(esM2.componentModelingDict['SourceSinkModel'],'optSummary',to_esm)
esM2.componentModelingDict['SourceSinkModel'].optSummary

NameError: name 'to_esm' is not defined

In [ ]:
dict_ = xr_dss['SourceSinkModel']['Electricity market']
#pd.DataFrame.from_dict(dict_, orient='index').stack(.to_frame())
to_esm = dict_.drop('operationVariablesOptimum').drop('time').to_dataframe().T
to_esm

space,Electrolyzer_Location,IndustryLocation
TAC,1.520486e+06,0.0
commodCosts,1.895914e+06,NaN
commodRevenues,3.754286e+05,NaN
operation,7.508571e+07,NaN


In [ ]:
pd.MultiIndex.fro

In [ ]:
xr_dss['SourceSinkModel']['Electricity market'].to_dataframe().set_index(['TAC'])

TAC   commodCosts  commodRevenues  \
space                 time                                               
Electrolyzer_Location 0     1.520486e+06  1.895914e+06   375428.571429   
                      1     1.520486e+06  1.895914e+06   375428.571429   
                      2     1.520486e+06  1.895914e+06   375428.571429   
                      3     1.520486e+06  1.895914e+06   375428.571429   
IndustryLocation      0     0.000000e+00           NaN             NaN   
                      1     0.000000e+00           NaN             NaN   
                      2     0.000000e+00           NaN             NaN   
                      3     0.000000e+00           NaN             NaN   

                               operation  operationVariablesOptimum  
space                 time                                           
Electrolyzer_Location 0     7.508571e+07               1.877143e+07  
                      1     7.508571e+07               3.754286e+07  
                      2     7.508571e+07               0.000000e+00  
                      3     7.508571e+07               1.877143e+07  
IndustryLocation      0              NaN                        NaN  
                      1              NaN                        NaN  
                      2              NaN                        NaN  
                      3              NaN                        NaN

In [ ]:
xr_dss = esm_datasets['Results']

#pd.DataFrame.from_dict(xr_dss['SourceSinkModel'])
for key in xr_dss['SourceSinkModel']['optSummary']:
    comp_dict = xr_dss['SourceSinkModel']['optSummary'][key]
    

comp_dict
xr_dss['SourceSinkModel']['optSummary']

KeyError: 'optSummary'

In [ ]:
# xr_dataset['ts_operationRateMax'].loc['Source, Wind (onshore)', :, :]
esm_datasets["Input"]["Sink"]["Industry site"]["ts_operationRateFix"].to_dataframe().unstack()

ts_operationRateFix                 
space Electrolyzer_Location IndustryLocation
time                                        
0                       0.0       13140000.0
1                       0.0       13140000.0
2                       0.0       13140000.0
3                       0.0       13140000.0

In [ ]:
esm_datasets["Results"]["SourceSinkModel"]["Electricity market"]

KeyError: 'Electricity market'

In [ ]:
esm_datasets["Parameters"]

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    locations:           {'IndustryLocation', 'Electrolyzer_Location'}
    commodities:         {'electricity', 'hydrogen'}
    commodityUnitsDict:  {'electricity': 'kW$_{el}$', 'hydrogen': 'kW$_{H_{2}...
    numberOfTimeSteps:   4
    hoursPerTimeStep:    2190
    costUnit:            1 Euro
    lengthUnit:          km
    verboseLogLevel:     1
    balanceLimit:        None
    lowerBound:          False

Or save it directly to NetCDF with `esm_to_netcdf`:

In [ ]:
_ = xrIO.esm_to_netcdf(esM, outputFileName="my_esm.nc")

 C:\Users\p.dunkel\.conda\envs\europe_ph_fine_save2\lib\site-packages\fine-2.2.1-py3.6.egg\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 C:\Users\p.dunkel\.conda\envs\europe_ph_fine_save2\lib\site-packages\fine-2.2.1-py3.6.egg\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 C:\Users\p.dunkel\.conda\envs\europe_ph_fine_save2\lib\site-packages\fine-2.2.1-py3.6.egg\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 C:\Users\p.dunkel\.conda\envs\europe_ph_fine_save2\lib\site-packages\fine-2.2.1-py3.6.egg\FINE\IOManagement\utilsIO.py:37: UserWarning:More than one _ found in series index.             Therefore, _ is not used to split the index
 C:\Users\p.dunkel\.conda\envs\europe_ph_fine_save2\lib\site-packages\fine-2

ValueError: Unable to update size for existing dimension'space' (4 != 2)

#### STEP 3. Load esM from NetCDF file or xarray datasets

You can load an esM from file with `netcdf_to_esm`.

In [ ]:
# esm_from_file = xrIO.netcdf_to_esm("my_esm.nc")  # Not implemented

In [ ]:
# esm_from_file.getComponentAttribute('Wind (onshore)', 'operationRateMax')

Or from datasets with `datasets_to_esm`.

In [ ]:
# Alternative to giving an xr dataset, you could pass the full path to your NETCDF file 
esm_from_datasets = xrIO.datasets_to_esm(esm_datasets)

In [ ]:
esm_from_datasets.getComponentAttribute('Industry site', 'operationRateFix')

In [ ]:
esm_datasets["Results"]["SourceSinkModel"]["Wind (onshore)"]